In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import glob
import csv
import os
from copy import deepcopy
import time
import random
import pickle

import matplotlib

## Function definitions

In [ ]:
def get_parameters(datadir):
    # Using readlines()
    file = open(datadir + 'parameters.dat', 'r')
    Lines = file.readlines()
    parameters = dict()
    for line in Lines:
        line = line.strip()
        spl_line = line.split(" ");
        if spl_line[0] in ["L", "T"]:
            parameters[spl_line[0]] = int(spl_line[1])
        elif spl_line[0] in ["p_A"]:
            parameters[spl_line[0]] = float(spl_line[1])
        elif spl_line[0] in ["COVID"]:
            C = int(spl_line[1])
            if C==1:
                parameters[spl_line[0]] = True
            else:
                parameters[spl_line[0]] = False
        else:
            parameters[spl_line[0]] = spl_line[1]
    return parameters

def get_arr_from_CSV(datadir, arrname, floating=False):
    with open(datadir + arrname + '.dat', newline='') as csvfile:
        S = list(csv.reader(csvfile))
        outarr = np.zeros((len(S),len(S[0]) ))
        col_cntr = 0
        for Si in S:
            col = Si
            for i in range(len(col)):
                if floating:
                    col[i] = float(col[i])
                else:
                    col[i] = int(col[i])
            outarr[col_cntr,:] = np.array(col)
            col_cntr += 1
        if len(S)==1:
            outarr = outarr[0,:]
    return outarr

def get_arr_from_CSV_lastval(datadir, arrname, floating=False):
    with open(datadir + arrname + '.dat', newline='') as csvfile:
        col = list(csv.reader(csvfile))[-1]
        for i in range(len(col)):
            if floating:
                out = float(col[i])
            else:
                out = int(col[i])
    return out

In [ ]:
# Generate the command to simulate a system with the following parameters:
R=5
L=1000
p_a=0.003 # 0.3%
T=2000

print(f"Simulate by running the command \n./NOVAa {L} {T} {p_a} {R}")

## Plotting timeseries of N, O, V, A and a states

In [ ]:
# Input data for timeseries plot
indir = 'data/data_p_350_R_5_3887208924641438126/'# R = 5, p_a = 0.35%

%matplotlib inline

fig, (ax) = plt.subplots(1, 1, dpi=175, figsize=(5.2,3.0))
plt.style.use('ggplot')

font = {'family' : 'sans',
        'weight' : 'normal',
        #'size'   : 16}
        'size'   : 12}

plt.rc('font', **font)

params = get_parameters(indir)
print(params)

NAR = get_arr_from_CSV(indir, 'NAR', floating=False)
OAR = get_arr_from_CSV(indir, 'OAR', floating=False)
VAR = get_arr_from_CSV(indir, 'VAR', floating=False)
AAR = get_arr_from_CSV(indir, 'AAR', floating=False)
aAR = get_arr_from_CSV(indir, 'aAR', floating=False)

cutoff_idx = 1000

NAR = NAR[:cutoff_idx]
OAR = OAR[:cutoff_idx]
VAR = VAR[:cutoff_idx]
AAR = AAR[:cutoff_idx]
aAR = aAR[:cutoff_idx]

cmap = matplotlib.cm.get_cmap('inferno')

colO = cmap(0/4)
colV = cmap(1/4)
cola = cmap(2/4)
colA = cmap(3/4)
colN = cmap(3.7/4)

plt.plot(NAR/params["L"]**2, label=r"$N$ density", color=colN)
plt.plot(OAR/params["L"]**2, label=r"$O$ density", color=colO)
plt.plot(VAR/params["L"]**2, label=r"$V$ density", color=colV)
plt.plot(aAR/params["L"]**2, label=r"$a$ density", color=cola)
plt.plot(AAR/params["L"]**2, label=r"$A$ density", color=colA)

plt.legend(loc='center left')
plt.ylim([-0.01,1.0065])
plt.xlim([0,cutoff_idx])
if params["p_A"]>0.03 or params["p_A"]==0:
    plt.title(r'$R=$' + f'{params["R_N"]}  ' + r"$p_a=$" + f'{int(np.round(100*params["p_A"]))}%')
else:
    plt.title(r'$R=$' + f'{params["R_N"]}  ' + r"$p_a=$" + f'{np.round(100*params["p_A"],2)}%')

## Plotting final state for range of p_a values, to identify threshold:

In [ ]:
## Load data:

In [ ]:
NARdict = dict()
OARdict = dict()
VARdict = dict()
AARdict = dict()
aARdict = dict()


# Input data directories, using regex/wildcards/glob:
datadirs = glob.glob("./data/*_R_1_*")

print("Directories:", len(datadirs))

ctr = 0
ctrmax = len(datadirs)
realtime0 = time.time()
for dd in datadirs:
    proc_progress = round(100*ctr/ctrmax)
    proc_progress_next = round(100*(ctr+1)/ctrmax)
    if proc_progress_next > proc_progress:
        realtime = time.time()
        drt = (realtime-realtime0)
        fracdone = ctr/ctrmax
        fracleft = 1-fracdone
        ETA = drt*fracleft/(fracdone+0.000001)
        print("Progress:", proc_progress_next,"%. ETA:", ETA, f"({round(ETA/60,1)} minutes).")
    # Check if empty:
    if not os.path.exists(dd + "/parameters.dat"):
        pass
    else:
        dd = dd + "/"
        params = get_parameters(dd)
        pA_promille = int(round(100000*params["p_A"]))
        NAR = get_arr_from_CSV_lastval(dd, "NAR")
        OAR = get_arr_from_CSV_lastval(dd, "OAR")
        VAR = get_arr_from_CSV_lastval(dd, "VAR")
        AAR = get_arr_from_CSV_lastval(dd, "AAR")
        aAR = get_arr_from_CSV_lastval(dd, "aAR")
        if pA_promille in VARdict.keys():
            pass
        else:
            NARdict[pA_promille] = []
            OARdict[pA_promille] = []
            VARdict[pA_promille] = []
            AARdict[pA_promille] = []
            aARdict[pA_promille] = []
        NARdict[pA_promille].append(deepcopy(NAR)/(params["L"]**2))
        OARdict[pA_promille].append(deepcopy(OAR)/(params["L"]**2))
        VARdict[pA_promille].append(deepcopy(VAR)/(params["L"]**2))
        AARdict[pA_promille].append(deepcopy(AAR)/(params["L"]**2))
        aARdict[pA_promille].append(deepcopy(aAR)/(params["L"]**2))
    ctr += 1
print(VARdict.keys())
print(f"Based on {ctr} simulations.")
for key in VARdict.keys():
    marker = ""
    if np.mean(VARdict[key]) > 0.1:
        marker += "\t <---- \t HIGH"
    if key/100000 > 0.61:
        marker += " above classical threshold"
    print(f"{key/1000}% -- {np.mean(VARdict[key])} {marker}")

In [ ]:
## Plot data:

In [ ]:
plt.style.use('ggplot')

font = {'family' : 'sans',
        'weight' : 'normal',
        #'size'   : 16}
        'size'   : 12}

plt.rc('font', **font)

pas = []
Ns = []
Vs = []
As = []
aas = []

for pa_p in VARdict.keys():
    pas.append(pa_p/100000)
    Ns.append(np.mean(NARdict[pa_p]))
    Vs.append(np.mean(VARdict[pa_p]))
    As.append(np.mean(AARdict[pa_p]))
    aas.append(np.mean(aARdict[pa_p]))

%matplotlib inline

fig, (ax) = plt.subplots(1, 1, dpi=175, figsize=(5.2,3.0))
plt.style.use('ggplot')

font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 12}

plt.rc('font', **font)

my_cmap = plt.cm.get_cmap('inferno')
Ocol = my_cmap(0)
Vcol = my_cmap(0.35)
acol = my_cmap(0.50)
Acol = my_cmap(0.70)
Ncol = my_cmap(0.86)

sortidx = np.argsort(pas)
pas = np.array(pas)
Ns = np.array(Ns)
Vs = np.array(Vs)
As = np.array(As)
aas = np.array(aas)
# Sorting
pas = pas[sortidx]
Ns = Ns[sortidx]
Vs = Vs[sortidx]
As = As[sortidx]
aas = aas[sortidx]



plt.plot(100*pas,1-(Ns+Vs+As), label=r"$O+a$", zorder=2, color=Ocol)


plt.plot(100*pas,Ns, "--", label=r"$N$", zorder=2, color=Ncol)
plt.plot(100*pas,Vs, "--", label=r"$V$", zorder=2, color=Vcol)


plt.plot(100*pas,As, label=r"$A$", alpha=1, zorder=1, color=Acol)


plt.plot(100*pas,aas, label=r"$a$", alpha=1, zorder=1, color=acol)

plt.ylim([-0.02,1.02])

plt.ylabel(r"Fraction of cells")
__ = plt.xlabel(r"$p_a$ (%)")
plt.legend()